## logistic regression model

#### import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### load cleaned feature data table

In [2]:
df = pd.read_feather('../data/processed/df_to_model')

In [3]:
df.columns

Index(['time_on_vent', 'anchor_age', 'spontrr', 'heartrate', 'std_spontrr',
       'weight', 'bloodpressure', 'std_pulseox', 'std_heartrate', 'height',
       'tidalvolume', 'temp', 'std_bloodpressure', 'pulseox', 're_intub_class',
       'gender', 'admission_type'],
      dtype='object')

In [4]:
df[df['re_intub_class']==0]

,time_on_vent,anchor_age,spontrr,heartrate,std_spontrr,weight,bloodpressure,std_pulseox,std_heartrate,height,tidalvolume,temp,std_bloodpressure,pulseox,re_intub_class,gender,admission_type
0,25.983333,72.0,19.0,79.0,3.718759,123.2,62.0,0.000000,1.971222,59.921260,387.0,36.444444,20.347548,100.0,0,F,EW EMER.
1,17.000000,23.0,33.0,122.0,3.577709,264.0,101.0,1.788854,13.771952,70.933071,527.0,39.166667,6.350853,96.0,0,M,EW EMER.
2,580.233333,81.0,24.0,63.0,1.788854,198.0,79.0,1.673320,4.324350,61.905512,437.0,36.833333,6.418723,96.0,0,F,EW EMER.
3,170.000000,65.0,15.0,80.0,2.316607,205.9,105.0,0.577350,3.633180,61.905512,614.0,37.333333,4.505552,100.0,0,F,URGENT
4,83.866667,61.0,18.0,73.0,8.455767,199.5,87.0,0.000000,7.694154,66.070866,556.0,36.444444,7.366591,100.0,0,F,EW EMER.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6783,22.216667,74.0,15.0,60.0,4.919350,121.0,98.0,0.000000,2.081666,66.070866,390.0,36.888889,3.271085,100.0,0,F,URGENT
6784,143.283333,59.0,21.0,88.0,2.035401,233.0,85.0,1.516575,7.582875,66.070866,334.0,36.555556,10.549882,96.0,0,M,EW EMER.
6785,21.216667,57.0,19.0,83.0,4.163332,255.2,78.0,1.032796,2.041241,68.501969,401.0,37.333333,3.405877,96.0,0,F,ELECTIVE
6786,8.550000,61.0,19.0,50.0,10.392305,197.1,94.0,0.983192,2.994439,70.039370,482.0,36.222222,5.656854,97.0,0,M,EW EMER.


## Handle categoricals

In [5]:
X = df[df.columns.drop('re_intub_class')]
y = df['re_intub_class']

In [6]:
# Import libraries 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [7]:
numeric_features  = df[df.columns.drop(['gender','admission_type','re_intub_class'])].columns
numeric_transformer = Pipeline(steps=[
   # ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = df[['gender','admission_type']].columns
categorical_transformer = Pipeline(steps=[
   # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('smote', SMOTE(k_neighbors = 5,random_state=101)),
                      ('classifier', LogisticRegression())])

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
]

svc_param_grid ={ 'svc__kernel':('linear', 'rbf'), 
                 'svc__C': [0.1,1, 10, 100, 1000], 
                 'svc__gamma': [1,0.1,0.01,0.001,0.0001]}

gs = GridSearchCV(clf, svc_param_grid, verbose=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 101)

clf.fit(X_train, y_train)
#print("model score: %.3f" % clf.score(X_test, y_test))

predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.93      0.61      0.73      1355
           1       0.13      0.56      0.22       148

    accuracy                           0.60      1503
   macro avg       0.53      0.58      0.48      1503
weighted avg       0.85      0.60      0.68      1503

[[822 533]
 [ 65  83]]


In [8]:
predictions = clf.predict(X_test)

#### 5. Evaulate model

In [9]:
from sklearn.metrics import classification_report, confusion_matrix

In [10]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.93      0.61      0.73      1355
           1       0.13      0.56      0.22       148

    accuracy                           0.60      1503
   macro avg       0.53      0.58      0.48      1503
weighted avg       0.85      0.60      0.68      1503



In [11]:
print(confusion_matrix(y_test,predictions))

[[822 533]
 [ 65  83]]


In [12]:
import joblib

In [13]:
#joblib.dump(logmodel, "reintubate_model_log.sav")

In [14]:
#joblib.dump(scaler, "reintubate_scaler.sav")

## Train model

### perform train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### 3. SMOTE IT!

from collections import Counter
counter = Counter(y_train)
print(counter)

# transform the dataset
oversample = SMOTE(random_state = 101)

X_smote, y_smote = oversample.fit_resample(X_train, y_train)

counter = Counter(y_smote)
print(counter)

#### 4. Do logistic regression model

from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(max_iter=1000, C=0.0001)
logmodel.fit(X_smote,y_smote)

import pickle

In [15]:
# Save the model as a pickle in a file 
#pickle.dump(logmodel, open("reintubate_model_log", 'wb')) 

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features  = df[df.columns.drop(['gender','admission_type','re_intub_class'])].columns
#numeric_transformer = ('scaler', StandardScaler())
numeric_transformer = make_column_transformer(
    (StandardScaler(), numeric_features),
    remainder='passthrough')


categorical_features = df[['gender','admission_type']].columns
#categorical_transformer =  ('onehot', OneHotEncoder(drop='first'))
categorical_transformer = make_column_transformer(
    (OneHotEncoder(drop='first'), categorical_features),
    remainder='passthrough')

#preprocessor = make_column_transformer(
 #   transformers=[
  #      ('num', numeric_transformer, numeric_features),
   #     ('cat', categorical_transformer, categorical_features)],
#remainder ='passthrough')

clf = Pipeline(steps=[('num', numeric_transformer),
                      ('cat', categorical_transformer),
                      ('classifier', LogisticRegression())])

#### 2. Perform feature scaling

Because the range of values in the features are not necessarily in the same order of magnitude, we will scale the feature data prior to training the model.

* actually... they might not be far off! 

mask=['spontRR', 'stdABP', 'meanABP', 'stdSpontRR', 'pulseox', 'stdPulseox',
       'temp', 'heartRate', 'stdHeartRate', 'weight', 'height', 'anchor_age',
       'time_on_vent']

X_traina = X_train.copy()
X_testa = X_test.copy()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train.loc[:,mask])
X_traina.loc[:,mask] = scaler.transform(X_train.loc[:,mask])

#X_train = scaler.transform(X_train)
X_testa.loc[:,mask] = scaler.transform(X_test.loc[:,mask])